<a href="https://colab.research.google.com/github/Ashishkhurana01/Hands-On-LLM/blob/main/ch03_LookingInsideLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Looking Inside Transformer LLMs

In [1]:
%%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda


In [3]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in


In [4]:
prompt = "The capital of France is"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Tokenize the input prompt
input_ids = input_ids.to("cuda")

# Get the output of the model before the lm_head
model_output = model.model(input_ids)

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])

In [5]:
input_ids

tensor([[ 450, 7483,  310, 3444,  338]], device='cuda:0')

In [7]:
print(model_output)

BaseModelOutputWithPast(last_hidden_state=tensor([[[-0.3047,  1.1953,  0.2988,  ..., -0.3008,  0.6758,  0.1406],
         [-0.1338,  0.3184,  0.3926,  ...,  0.5508, -0.1318, -0.6055],
         [-0.5820,  1.0703,  1.5547,  ..., -0.4082,  0.2773,  0.4062],
         [-0.3984,  0.6719,  0.2539,  ..., -0.0732,  0.8164, -0.6875],
         [-0.6758,  0.7070,  0.5469,  ...,  0.2598,  0.1621, -0.2773]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MulBackward0>), past_key_values=((tensor([[[[-2.9688e-01,  1.3770e-01, -7.8125e-02,  ..., -9.1797e-02,
           -2.6123e-02,  5.8350e-02],
          [ 8.6914e-02,  9.7656e-02, -1.3672e-01,  ..., -2.3926e-02,
           -1.0840e-01, -2.7100e-02],
          [ 4.0820e-01,  4.1504e-02, -1.9141e-01,  ..., -1.5137e-01,
           -2.1094e-01,  8.7891e-02],
          [ 2.5977e-01, -3.2227e-02, -3.8330e-02,  ..., -1.6602e-02,
           -1.2268e-02,  6.3782e-03],
          [ 9.8633e-02, -9.8633e-02, -7.1289e-02,  ..., -1.9336e-01,
            2.3

In [10]:
print(lm_head_output)

tensor([[[24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
         [31.1250, 31.5000, 26.0000,  ..., 26.0000, 26.0000, 26.0000],
         [31.5000, 28.8750, 31.1250,  ..., 26.2500, 26.2500, 26.2500],
         [33.0000, 31.8750, 36.0000,  ..., 27.8750, 27.8750, 27.8750],
         [27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)


In [11]:
print(lm_head_output[0,-1])

tensor([27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)


In [ ]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

In [ ]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [ ]:
lm_head_output.shape

torch.Size([1, 5, 32064])